## 1. Setup and Data Loading

In [ ]:
# Import libraries
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

from scripts.data_loader import DataLoader
from scripts.preprocessing import DataPreprocessor
from scripts.visualizations import InsuranceVisualizer

warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("Libraries imported successfully!")

In [ ]:
# Load data
# Note: Update the path to your actual data file
DATA_PATH = '../data/MachineLearningRating_v3.txt'  # or .csv

loader = DataLoader(DATA_PATH)
df = loader.load_data()

print(f"\nData Shape: {df.shape}")
print(f"Columns: {len(df.columns)}")
print(f"Rows: {len(df)}")

## 2. Data Understanding

In [ ]:
# Display first few rows
df.head(10)

In [ ]:
# Data info
info = loader.get_data_info()
print("\n=== Data Information ===")
print(f"Shape: {info['shape']}")
print(f"Memory Usage: {info['memory_usage']:.2f} MB")
print(f"Duplicates: {info['duplicates']}")

In [ ]:
# Column types
col_types = loader.get_column_types()
print("\n=== Column Types ===")
print(f"Numerical columns: {len(col_types['numerical'])}")
print(f"Categorical columns: {len(col_types['categorical'])}")
print(f"Datetime columns: {len(col_types['datetime'])}")

In [ ]:
# Data types
df.dtypes

## 3. Data Quality Assessment

In [ ]:
# Missing values
missing = df.isnull().sum()
missing_pct = (missing / len(df)) * 100
missing_df = pd.DataFrame({
    'Missing_Count': missing,
    'Percentage': missing_pct
}).sort_values('Percentage', ascending=False)

print("\n=== Missing Values ===")
print(missing_df[missing_df['Percentage'] > 0].head(20))

In [ ]:
# Visualize missing values
visualizer = InsuranceVisualizer(df)
visualizer.create_missing_values_plot()

## 4. Descriptive Statistics

In [ ]:
# Summary statistics for key numerical columns
key_cols = ['TotalPremium', 'TotalClaims', 'SumInsured', 'CalculatedPremiumPerTerm']
key_cols = [col for col in key_cols if col in df.columns]

df[key_cols].describe()

In [ ]:
# Overall statistics
print("\n=== Overall Portfolio Statistics ===")
print(f"Total Policies: {len(df):,}")
print(f"Total Premium: R {df['TotalPremium'].sum():,.2f}")
print(f"Total Claims: R {df['TotalClaims'].sum():,.2f}")
print(f"Overall Loss Ratio: {(df['TotalClaims'].sum() / df['TotalPremium'].sum()):.4f}")

## 5. Data Preprocessing

In [ ]:
# Preprocess data
preprocessor = DataPreprocessor(df)
df_clean = preprocessor.convert_data_types()
df_clean = preprocessor.create_features()

print("\n=== Preprocessing Summary ===")
print(preprocessor.get_preprocessing_summary())

In [ ]:
# Check new features
new_features = ['LossRatio', 'ProfitMargin', 'HasClaim', 'VehicleAge']
new_features = [f for f in new_features if f in df_clean.columns]

df_clean[new_features].describe()

## 6. Univariate Analysis

In [ ]:
# Distribution of Total Premium
visualizer = InsuranceVisualizer(df_clean)
visualizer.plot_distribution('TotalPremium')

In [ ]:
# Distribution of Total Claims
visualizer.plot_distribution('TotalClaims')

In [ ]:
# Distribution of Loss Ratio
if 'LossRatio' in df_clean.columns:
    visualizer.plot_distribution('LossRatio')

In [ ]:
# Categorical analysis - Province
if 'Province' in df_clean.columns:
    visualizer.plot_categorical('Province')

In [ ]:
# Categorical analysis - Vehicle Type
if 'VehicleType' in df_clean.columns:
    visualizer.plot_categorical('VehicleType')

In [ ]:
# Categorical analysis - Gender
if 'Gender' in df_clean.columns:
    visualizer.plot_categorical('Gender')

## 7. Bivariate Analysis

In [ ]:
# Claims vs Premium scatter plot
visualizer.plot_claims_vs_premium()

In [ ]:
# Correlation matrix
corr_cols = ['TotalPremium', 'TotalClaims', 'SumInsured', 'LossRatio', 
             'ProfitMargin', 'VehicleAge']
corr_cols = [col for col in corr_cols if col in df_clean.columns]

visualizer.plot_correlation_matrix(corr_cols)

## 8. Loss Ratio Analysis

In [ ]:
# Loss Ratio by Province
if 'Province' in df_clean.columns and 'LossRatio' in df_clean.columns:
    visualizer.plot_loss_ratio_by_category('Province')

In [ ]:
# Loss Ratio by Vehicle Type
if 'VehicleType' in df_clean.columns and 'LossRatio' in df_clean.columns:
    visualizer.plot_loss_ratio_by_category('VehicleType')

In [ ]:
# Loss Ratio by Gender
if 'Gender' in df_clean.columns and 'LossRatio' in df_clean.columns:
    loss_by_gender = df_clean.groupby('Gender').agg({
        'TotalClaims': 'sum',
        'TotalPremium': 'sum',
        'LossRatio': 'mean',
        'PolicyID': 'count'
    })
    loss_by_gender.columns = ['Total_Claims', 'Total_Premium', 'Avg_Loss_Ratio', 'Policy_Count']
    print("\n=== Loss Ratio by Gender ===")
    print(loss_by_gender)

## 9. Geographic Analysis

In [ ]:
# Geographic analysis by Province
if 'Province' in df_clean.columns:
    visualizer.plot_geographic_analysis('Province')

In [ ]:
# Top 10 Postal Codes by Loss Ratio
if 'PostalCode' in df_clean.columns and 'LossRatio' in df_clean.columns:
    top_postal = df_clean.groupby('PostalCode').agg({
        'LossRatio': 'mean',
        'TotalPremium': 'sum',
        'TotalClaims': 'sum',
        'PolicyID': 'count'
    }).sort_values('LossRatio', ascending=False).head(10)
    
    print("\n=== Top 10 Postal Codes by Loss Ratio ===")
    print(top_postal)

## 10. Temporal Trends

In [ ]:
# Temporal trends
if 'TransactionMonth' in df_clean.columns:
    visualizer.plot_temporal_trends('TransactionMonth')

## 11. Vehicle Analysis

In [ ]:
# Top vehicle makes by claim amount
if 'Make' in df_clean.columns:
    top_makes = df_clean.groupby('Make').agg({
        'TotalClaims': 'sum',
        'TotalPremium': 'sum',
        'PolicyID': 'count'
    }).sort_values('TotalClaims', ascending=False).head(10)
    
    print("\n=== Top 10 Vehicle Makes by Total Claims ===")
    print(top_makes)

In [ ]:
# Vehicle Age analysis
if 'VehicleAge' in df_clean.columns:
    age_analysis = df_clean.groupby(pd.cut(df_clean['VehicleAge'], bins=[-1, 5, 10, 15, 20, 100])).agg({
        'LossRatio': 'mean',
        'TotalClaims': 'mean',
        'PolicyID': 'count'
    })
    
    print("\n=== Analysis by Vehicle Age Groups ===")
    print(age_analysis)

## 12. Outlier Detection

In [ ]:
# Detect outliers in key columns
def detect_outliers_iqr(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 3 * IQR
    upper_bound = Q3 + 3 * IQR
    outliers = ((data[column] < lower_bound) | (data[column] > upper_bound)).sum()
    return outliers, lower_bound, upper_bound

print("\n=== Outlier Detection ===")
for col in ['TotalPremium', 'TotalClaims', 'LossRatio']:
    if col in df_clean.columns:
        outliers, lower, upper = detect_outliers_iqr(df_clean, col)
        print(f"\n{col}:")
        print(f"  Outliers: {outliers} ({outliers/len(df_clean)*100:.2f}%)")
        print(f"  Bounds: [{lower:.2f}, {upper:.2f}]")

## 13. Key Insights Summary

In [ ]:
print("\n" + "="*60)
print("KEY INSIGHTS FROM EDA")
print("="*60)

# 1. Overall Portfolio Performance
print("\n1. OVERALL PORTFOLIO PERFORMANCE")
total_premium = df_clean['TotalPremium'].sum()
total_claims = df_clean['TotalClaims'].sum()
overall_loss_ratio = total_claims / total_premium
print(f"   - Overall Loss Ratio: {overall_loss_ratio:.4f}")
print(f"   - Total Premium: R {total_premium:,.2f}")
print(f"   - Total Claims: R {total_claims:,.2f}")
print(f"   - Total Policies: {len(df_clean):,}")

# 2. Provincial Analysis
if 'Province' in df_clean.columns:
    print("\n2. PROVINCIAL ANALYSIS")
    province_stats = df_clean.groupby('Province').agg({
        'LossRatio': 'mean',
        'PolicyID': 'count'
    }).sort_values('LossRatio')
    print(f"   - Lowest Loss Ratio: {province_stats.index[0]} ({province_stats['LossRatio'].iloc[0]:.4f})")
    print(f"   - Highest Loss Ratio: {province_stats.index[-1]} ({province_stats['LossRatio'].iloc[-1]:.4f})")

# 3. Gender Analysis
if 'Gender' in df_clean.columns:
    print("\n3. GENDER ANALYSIS")
    gender_stats = df_clean.groupby('Gender')['LossRatio'].mean()
    for gender, ratio in gender_stats.items():
        print(f"   - {gender}: {ratio:.4f}")

# 4. Data Quality
print("\n4. DATA QUALITY")
print(f"   - Missing Values: {df_clean.isnull().sum().sum()}")
print(f"   - Duplicate Rows: {df_clean.duplicated().sum()}")

print("\n" + "="*60)

## 14. Save Processed Data

In [ ]:
# Save processed data for further analysis
# output_path = '../data/processed/insurance_data_processed.csv'
# df_clean.to_csv(output_path, index=False)
# print(f"\nProcessed data saved to: {output_path}")

## Next Steps

1. **A/B Hypothesis Testing** - Test specific hypotheses about risk differences
2. **Statistical Modeling** - Build predictive models for claims and premiums
3. **Feature Engineering** - Create additional features for modeling
4. **Business Recommendations** - Translate insights into actionable strategies